In [ ]:
# %%script false --no-raise-error
!rm -fr './kaggle/'


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import fastai
import dill
from fastai.imports import *
from fastai.vision import *
from fastai.vision.all import *
from fastai.metrics import error_rate, accuracy

In [ ]:
## Defining Directories
path = Path('../input/hotel-id-2021-fgvc8')

## csv
sample_sub = path/'sample_submission.csv'
labels = path/'train.csv'

In [ ]:
train_data = pd.read_csv(labels)
train_data.head(5)

In [ ]:
!tar xzvf /kaggle/input/hotelid-224/kaggle_hotelid_train_imgs_224x224.tgz

In [ ]:
train_dir = './kaggle/working/train/'

train_data['image_path'] = train_dir + train_data.chain.astype('str') + '/' + train_data.image
train_data[:5]

## Creating a data Loader of the entire Dataset

In [ ]:
batch_size = 32
batch_size = 32
def create_dataLoader():
    dataset = ImageDataLoaders.from_df(df = train_data[['image_path', 'hotel_id']],
                                   path = '.',
                                   folder = '.',
                                   valid_pct=0.2,
                                   item_tfms=Resize(224, method='pad', pad_mode='reflection'),
                                   batch_tfms=aug_transforms(size=224),
                                   bs=batch_size)
    return dataset

In [ ]:
dataset = create_dataLoader()

In [ ]:
dataset.show_batch()

## Training dataset

In [ ]:
%%script false --no-raise-error
def training_models(dataset, model_name):
    learn = cnn_learner(dataset,  models.resnet101, 
                        metrics=[accuracy, top_k_accuracy], 
                        opt_func=QHAdam).to_fp16()
    learn.fine_tune(5, 0.005, freeze_epochs=3)
    learn.save(model_name)
    return learn

In [ ]:
%%script false --no-raise-error
%time
model_name ='model_allData_rn50'
learn = training_models(dataset, model_name)

## Exporting the model


In [ ]:
%%script false --no-raise-error
learn.export(f'{model_name}.pkl', pickle_module=dill)

In [ ]:
%%script false --no-raise-error
fromDir ='../input/hotelid-model/model_Data_category_rn101.pkl'
toDir ='.'
copy_tree(fromDir, toDir)

## Loading the model

In [ ]:
model_path ='../input/hotelid-model/model_Data_category_rn101.pkl'
learn = load_learner(fname = Path(model_path), cpu=False, pickle_module=dill) 

## Predictions

In [ ]:
sample_submission = pd.read_csv('../input/hotel-id-2021-fgvc8/sample_submission.csv')
submission = sample_submission.copy()
submission['image'] = '../input/hotel-id-2021-fgvc8/test_images/' + submission['image']
submission

In [ ]:
test_dl = learn.dls.test_dl(submission)

In [ ]:
test_dl.show_batch() 

In [ ]:
probs, _ = learn.tta(dl=test_dl)

In [ ]:
preds_idx = probs.topk(5)[1]

In [ ]:
preds = [' '.join(map(str, learn.dls.vocab[pred])) for pred in preds_idx]
preds[:5]

In [ ]:
os.chdir(".")
submission = sample_submission.copy()
submission['hotel_id'] = preds

submission.to_csv('submission.csv', index = False)

submission.head()

In [ ]:
!rm -fr './kaggle/working/train'